In [3]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from langchain.tools import tool
from collections import defaultdict
from dotenv import load_dotenv
from typing import List, Dict, Optional, Union, Any, Tuple
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from bs4 import BeautifulSoup
from langchain_community.tools import DuckDuckGoSearchRun
import difflib
import time
import requests
import urllib.parse
import re
import os

In [4]:
load_dotenv()
LLM = ChatOpenAI(model = "gpt-4.1")
search = TavilySearchResults() # will be used to search the web

search_2 = DuckDuckGoSearchRun()

In [14]:
def fetch_stats(url: str, headers: dict) -> dict:
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
       return {}
    return response.json()

In [15]:
# TOOLS
@tool
def match_info():
    """chghmhnfgb"""
    url = "https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/matches/upcoming"
    headers = {
        'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM', # API KEY
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '1943a818-98e9-48ea-8d1c-1554e116ef44'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code}")
    
    data = response.json()
    ipl_match_list = []

    for type_match in data.get("typeMatches", []):
        for series_match in type_match.get("seriesMatches", []):
            series = series_match.get("seriesAdWrapper", {})
            if "Indian Premier League" in series.get("seriesName", ""):
                for match in series.get("matches", []):
                    match_info = match.get("matchInfo", {})
                    match_id = match_info.get("matchId")
                    match_desc = match_info.get("matchDesc")
                    match_status = match_info.get("status")
                    team1 = match_info.get("team1", {}).get("teamName", "Team 1")
                    team2 = match_info.get("team2", {}).get("teamName", "Team 2")
                    venue = match_info.get("venueInfo", {})
                    venue_id = venue.get("id", "Unknown ID")
                    ground = venue.get("ground", "Unknown Ground")
                    city = venue.get("city", "Unknown City")

                    ipl_match_list.append({
                        "Match ID": match_id,
                        "Match Desc": match_desc,
                        "Teams": f"{team1} vs {team2}",
                        "Status": match_status,
                        "Venue ID": venue_id,
                        "Venue": f"{ground}, {city}"
                    })
    
    return ipl_match_list

@tool
def additional_info(match_id: str) -> str: # about pitch, probable players, injuries about ground,
    """zxcv"""
    url = f"https://Cricbuzz-Official-Cricket-API.proxy-production.allthingsdev.co/match/{match_id}/commentary"
    headers = {
        'x-apihub-key': '9HN92wz6l7bberNNuKkhDCXeb4YH4lXo2fIKuVdgCpB82jpHlM',
        'x-apihub-host': 'Cricbuzz-Official-Cricket-API.allthingsdev.co',
        'x-apihub-endpoint': '8cb69a0f-bcaa-45b5-a016-229a2e7594f6'
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to get commentary: {response.status_code}")

    data = response.json()
    full_text = ""

    # Concatenate all commText entries
    for item in data.get("commentaryList", []):
        full_text += item.get("commText", "") + " "

    # Remove ALL markers like B0$, B1$, B14$ (anywhere in the text)
    cleaned = re.sub(r'\s*B\d+\$', '', full_text)
    
    # Remove escaped newlines and excess spaces
    cleaned = cleaned.replace("\\n", " ")
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    return cleaned

In [16]:
# 1. Researcher Agent: gathers match details (teams, date, venue, weather, pitch, odds).
research_agent = create_react_agent(
    model = LLM,
    name = "researcher",
    tools = [search, match_info, additional_info],
    prompt = (
        "You are an agent which is phenomenal at maths. Answer the query of the user to the best you can."
    )
)

In [17]:
inputs = {"messages": [{"role": "user", "content": "what is the output when we add, multiply, divide, and find diff of 2 and 20"}]}
result = research_agent.invoke(inputs)
for r in result['messages']:
    print(r)
print(result["messages"][-1].content)

content='what is the output when we add, multiply, divide, and find diff of 2 and 20' additional_kwargs={} response_metadata={} id='aaca483e-cf13-4d8c-a5e8-fb80ea1a728c'
content="Let's perform the calculations step by step:\n\n- Addition: 2 + 20 = 22\n- Multiplication: 2 × 20 = 40\n- Division: 2 ÷ 20 = 0.1\n- Difference (Subtraction): 2 − 20 = -18\n\nSummary:\n- Sum: 22\n- Product: 40\n- Quotient: 0.1\n- Difference: -18" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 156, 'total_tokens': 248, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-Bf1qKZaN8TiplLzlfHz9tDw0UNDCu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} name='researcher' id='run--009d31eb-a

In [18]:
# 2 Agents More -> Strategizer, Selevctor
# https://allcric.com/blog/a-guide-to-analyzing-pitch-conditions-for-fantasy-cricket-success/

In [19]:



def _name_variants(full_name: str) -> list:
    """
    Generate possible abbreviated variants for a multi-word name.
    - If two words (First Last): ["First Last", "F Last"]
    - If three words (A B C): ["A B C", "AB C", "A BC"]
    Additional variants can be added as needed.
    """
    parts = full_name.strip().split()
    variants = [full_name.strip()]

    if len(parts) == 2:
        first, last = parts
        variants.append(f"{first[0]} {last}")
    elif len(parts) == 3:
        a, b, c = parts
        variants.append(f"{a[0]}{b[0]} {c}")  # e.g. "AB C"
        variants.append(f"{a[0]} {b} {c}")    # e.g. "A B C" → but already full

    return variants


def fetch_cricmetric_table_via_scrapedo(
    batsman: str, bowler: str
) -> str:
    """
    1. Tries full `batsman` vs `bowler` name pair on CricMetric via Scrape.do.
    2. If no <table> found, tries abbreviated variants (e.g., "F Last", "AB C").
    3. Filters for only the “T20I” and “TWENTY20” tables on the page (using the
       enclosing panel-heading text).
    4. Returns raw HTML consisting of those filtered <table class="table">…</table>
       blocks concatenated, or "" if none matched.
    """
    base_matchup = "https://www.cricmetric.com/matchup.py"

    def attempt_fetch(name_a: str, name_b: str) -> str:
        """Attempt to fetch, then extract and return only T20I/TWENTY20 tables."""
        a_q = name_a.replace(" ", "+")
        b_q = name_b.replace(" ", "+")
        matchup_url = f"{base_matchup}?batsman={a_q}&bowler={b_q}&groupby=match"
        quoted = urllib.parse.quote(matchup_url, safe="")
        token = "c7cda0a41de3446abf92b8b0154c65e7922123609fe"
        scrape_do = f"http://api.scrape.do/?token={token}&url={quoted}&render=true"
        headers = {"User-Agent": "Mozilla/5.0"}
        resp = requests.get(scrape_do, headers = headers)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        filtered_html = []

        # 1) Find every panel that wraps an entire section (ODI, T20I, etc.)
        for panel in soup.find_all("div", class_="panel panel-default"):
            # 2) Read its heading text
            heading_div = panel.find("div", class_="panel-heading")
            label = heading_div.get_text(strip=True).upper() if heading_div else ""

            # 3) If that heading says "T20I" or "TWENTY20", grab its <table class="table">
            if "T20I" in label or "TWENTY20" in label:
                # Inside this panel, find the first <table class="table">
                tbl = panel.find("table", class_="table")
                if tbl:
                    filtered_html.append(str(tbl))

        # 4) Return all matched tables concatenated (or "" if none)
        return "".join(filtered_html)

    # First try full names
    table_html = attempt_fetch(batsman, bowler)
    if table_html:
        return table_html

    # No table for full names: generate and try variants
    bats_variants = _name_variants(batsman)
    bowl_variants = _name_variants(bowler)

    for bv in bats_variants:
        for ov in bowl_variants:
            if bv == batsman and ov == bowler:
                continue
            table_html = attempt_fetch(bv, ov)
            if table_html:
                return table_html

    # If none worked, return empty
    return ""


def parse_cricmetric_total_row(table_html: str, batsman: str, bowler: str) -> Dict[str, str]:
    """
    Given HTML containing one or more <table class="table"> blocks (already
    filtered to only T20I/TWENTY20 by fetch_cricmetric_table_via_scrapedo), this:
      1) Parses each table’s header row to get column names.
      2) Sums up <tbody> row counts across tables to get total “Innings.”
      3) Extracts each <tfoot><tr> “Total” row from every table, converts numeric cells,
         and aggregates them column‐wise.
      4) Computes combined Strike Rate and Average from aggregated Runs, Balls, Outs.
      5) Returns a dict:
         {
           "Title": "Batsman V/S Bowler",
           "Stats": {
             "Innings": "<total_innings>",
             "Runs": "<sum_of_runs>",
             "Balls": "<sum_of_balls>",
             "Outs": "<sum_of_outs>",
             "Dots": "<sum_of_dots>",
             "4s": "<sum_of_4s>",
             "6s": "<sum_of_6s>",
             "SR": "<computed_SR>",
             "Avg": "<computed_Avg>"
           }
         }
    """
    result: Dict[str, Any] = {}
    result["Title"] = f"{batsman} V/S {bowler}"
    stats: Dict[str, str] = {}

    soup = BeautifulSoup(table_html, "html.parser")
    tables = soup.find_all("table", class_="table")
    if not tables:
        raise RuntimeError("No <table class='table'> blocks found to extract totals.")

    # Extract headers from the first table
    first_header_row = tables[0].find("tr")
    if not first_header_row:
        raise RuntimeError("No <tr> found in first table to extract headers.")
    headers = [th.get_text(strip=True) for th in first_header_row.find_all("th")]

    # Initialize running totals for each numeric column
    running_totals: Dict[str, float] = {col: 0.0 for col in headers[1:]}
    total_innings = 0

    for tbl in tables:
        # Count how many <tr> exist inside <tbody> for this table
        tbody = tbl.find("tbody")
        if not tbody:
            continue
        body_rows = tbody.find_all("tr")
        total_innings += len(body_rows)

        # Extract the <tfoot><tr> from this table
        tfoot = tbl.find("tfoot")
        if not tfoot:
            continue
        total_row = tfoot.find("tr")
        if not total_row:
            continue
        cells = total_row.find_all("td")
        if len(cells) != len(headers):
            raise RuntimeError(
                f"Header count ({len(headers)}) != Total row cell count ({len(cells)})."
            )

        # Sum up this table’s totals into running_totals
        for col_name, cell in zip(headers[1:], cells[1:]):
            text = cell.get_text(strip=True).replace(",", "")
            try:
                val = float(text)
            except ValueError:
                val = 0.0
            running_totals[col_name] += val

    # Now compute combined SR and Avg from aggregated Runs, Balls, Outs
    total_runs = running_totals.get("Runs", 0.0)
    total_balls = running_totals.get("Balls", 0.0)
    total_outs = running_totals.get("Outs", 0.0)

    combined_sr = 0.0
    if total_balls > 0:
        combined_sr = (total_runs / total_balls) * 100.0

    combined_avg = 0.0
    if total_outs > 0:
        combined_avg = total_runs / total_outs

    # Populate stats dict
    stats["Innings"] = str(total_innings - 1)
    stats["Runs"] = str(int(running_totals.get("Runs", 0.0)))
    stats["Balls"] = str(int(running_totals.get("Balls", 0.0)))
    stats["Outs"] = str(int(running_totals.get("Outs", 0.0)))
    stats["Dots"] = str(int(running_totals.get("Dots", 0.0)))
    stats["4s"] = str(int(running_totals.get("4s", 0.0)))
    stats["6s"] = str(int(running_totals.get("6s", 0.0)))
    stats["SR"] = f"{combined_sr:.1f}"
    stats["Avg"] = f"{combined_avg:.1f}"

    result["Stats"] = stats
    return result


def players_faceoff(
    batsman: str, bowler: str
) -> Dict[str, str]:
    """
    High-level helper that:
      1) Calls `fetch_cricmetric_table_via_scrapedo(...)` to retrieve the <table> HTML.
      2) If no table HTML is returned, returns an empty dict.
      3) Otherwise calls `parse_cricmetric_total_row(...)` to extract the “Total” row + match count.
      4) Returns the combined dict.
    """
    table_html = fetch_cricmetric_table_via_scrapedo(batsman, bowler)
    if not table_html:
        return {}
    return parse_cricmetric_total_row(table_html, batsman, bowler)


In [19]:
print(head_2_head.invoke(
    {"team_A":[
        {
      "name": "Virat Kohli",
      "role": "batsman",
      "is_wk": "False",
      "is_overseas": "False",
      "batting_style": "Right Handed Bat",
      "bowling_style": "Right-arm medium",
      "recent_stats": [
        {
          "title": "last_8_innings_stats",
          "data": {
            "Batting": {
              "Matches": 8,
              "Innings": 8,
              "Runs": 408,
              "Balls": 278,
              "Outs": 7,
              "4s": 46,
              "6s": 9,
              "50s": 5,
              "100s": 0,
              "SR": 146.76,
              "Avg": 58.29
            }
          }
        },
        {
          "title": "career_stats_vs_Punjab_Kings",
          "data": {
            "Batting": {
              "Matches": 36,
              "Innings": 36,
              "Runs": 1159,
              "Balls": 874,
              "Outs": 32,
              "4s": 120,
              "6s": 33,
              "50s": 6,
              "100s": 1,
              "SR": 132.6,
              "Avg": 36.21
            }
          }
        },
        {
          "title": "career_stats_at_M_Chinnaswamy_Stadium",
          "data": {
            "Batting": {
              "Matches": 109,
              "Innings": 106,
              "Runs": 3618,
              "Balls": 2514,
              "Outs": 92,
              "4s": 329,
              "6s": 154,
              "50s": 27,
              "100s": 4,
              "SR": 143.91,
              "Avg": 39.32
            }
          }
        }
      ]
    },
     {
      "name": "Hardik Pandya",
      "role": "batting allrounder",
      "is_wk": "False",
      "is_overseas": "False",
      "batting_style": "Right Handed Bat",
      "bowling_style": "Right-arm fast-medium",
      "recent_stats": [
        {
          "title": "last_8_innings_stats",
          "data": {
            "Batting": {
              "Matches": 8,
              "Innings": 7,
              "Runs": 120,
              "Balls": 76,
              "Outs": 5,
              "4s": 9,
              "6s": 6,
              "50s": 0,
              "100s": 0,
              "SR": 157.89,
              "Avg": 24.0
            },
            "Bowling": {
              "Matches": 8,
              "Innings": 7,
              "Overs": 13.0,
              "Maidens": 0,
              "Runs": 146,
              "Wkts": 3,
              "Eco": 11.23,
              "Avg": 48.67,
              "SR": 26.0
            }
          }
        },
        {
          "title": "career_stats_vs_Royal_Challengers_Bengaluru",
          "data": {
            "Batting": {
              "Matches": 18,
              "Innings": 17,
              "Runs": 361,
              "Balls": 220,
              "Outs": 8,
              "4s": 22,
              "6s": 26,
              "50s": 2,
              "100s": 0,
              "SR": 164.09,
              "Avg": 45.12
            },
            "Bowling": {
              "Matches": 18,
              "innings": 12,
              "Overs": 29.0,
              "Maidens": 0,
              "Runs": 303,
              "Wkts": 7,
              "Eco": 10.44,
              "Avg": 43.28,
              "SR": 24.86
            }
          }
        },
        {
          "title": "career_stats_at_M_Chinnaswamy_Stadium",
          "data": {
            "Batting": {
              "Matches": 12,
              "Innings": 9,
              "Runs": 162,
              "Balls": 112,
              "Outs": 6,
              "4s": 12,
              "6s": 9,
              "50s": 1,
              "100s": 0,
              "SR": 144.64,
              "Avg": 27.0
            },
            "Bowling": {
              "Matches": 12,
              "innings": 10,
              "Overs": 27.0,
              "Maidens": 0,
              "Runs": 240,
              "Wkts": 11,
              "Eco": 8.88,
              "Avg": 21.81,
              "SR": 14.73
            }
          }
        }
      ]
    }
    ],
     "team_B": [{
      "name": "Jasprit Bumrah",
      "role": "bowler",
      "is_wk": "False",
      "is_overseas": "False",
      "batting_style": "Right Handed Bat",
      "bowling_style": "Right-arm fast",
      "recent_stats": [
        {
          "title": "last_8_innings_stats",
          "data": {
            "Bowling": {
              "Matches": 8,
              "Innings": 8,
              "Overs": 31.2,
              "Maidens": 0,
              "Runs": 197,
              "Wkts": 14,
              "Eco": 6.31,
              "Avg": 14.07,
              "SR": 13.43
            }
          }
        },
        {
          "title": "career_stats_vs_Royal_Challengers_Bengaluru",
          "data": {
            "Bowling": {
              "Matches": 20,
              "innings": 20,
              "Overs": 78.0,
              "Maidens": 2,
              "Runs": 581,
              "Wkts": 29,
              "Eco": 7.44,
              "Avg": 20.03,
              "SR": 16.14
            }
          }
        },
        {
          "title": "career_stats_at_M_Chinnaswamy_Stadium",
          "data": {
            "Bowling": {
              "Matches": 10,
              "innings": 10,
              "Overs": 37.3,
              "Maidens": 1,
              "Runs": 263,
              "Wkts": 14,
              "Eco": 7.01,
              "Overs": 78.0,
              "Maidens": 2,
              "Runs": 581,
              "Wkts": 29,
              "Eco": 7.44,
              "Avg": 20.03,
              "SR": 16.14
            }
          }
        }
      ]
    },
        {
      "name": "Shreyas Iyer",
      "role": "batsman",
      "is_wk": "False",
      "is_overseas": "False",
      "batting_style": "Right Handed Bat",
      "bowling_style": "Right-arm legbreak",
      "recent_stats": [
        {
          "title": "last_8_innings_stats",
          "data": {
            "Batting": {
              "Matches": 8,
              "Innings": 8,
              "Runs": 316,
              "Balls": 187,
              "Outs": 6,
              "4s": 25,
              "6s": 18,
              "50s": 3,
              "100s": 0,
              "SR": 168.98,
              "Avg": 52.67
            }
          }
        },
        {
          "title": "career_stats_vs_Royal_Challengers_Bengaluru",
          "data": {
            "Batting": {
              "Matches": 18,
              "Innings": 18,
              "Runs": 409,
              "Balls": 341,
              "Outs": 17,
              "4s": 34,
              "6s": 13,
              "50s": 4,
              "100s": 0,
              "SR": 119.94,
              "Avg": 24.05
            }
          }
        },
        {
          "title": "career_stats_at_M_Chinnaswamy_Stadium",
          "data": {
            "Batting": {
              "Matches": 11,
              "Innings": 11,
              "Runs": 305,
              "Balls": 222,
              "Outs": 9,
              "4s": 26,
              "6s": 14,
              "50s": 3,
              "100s": 0,
              "SR": 137.38,
              "Avg": 33.88
            }
          }
        }
      ]
    }
     ], 
     "pitch_cond": "seamer friendly"}))

([{'name': 'Virat Kohli', 'role': 'batsman', 'is_wk': 'False', 'is_overseas': 'False', 'batting_style': 'Right Handed Bat', 'bowling_style': 'Right-arm medium', 'recent_stats': [{'title': 'last_8_innings_stats', 'data': {'Batting': {'Matches': 8, 'Innings': 8, 'Runs': 408, 'Balls': 278, 'Outs': 7, '4s': 46, '6s': 9, '50s': 5, '100s': 0, 'SR': 146.76, 'Avg': 58.29}}}, {'title': 'career_stats_vs_Punjab_Kings', 'data': {'Batting': {'Matches': 36, 'Innings': 36, 'Runs': 1159, 'Balls': 874, 'Outs': 32, '4s': 120, '6s': 33, '50s': 6, '100s': 1, 'SR': 132.6, 'Avg': 36.21}}}, {'title': 'career_stats_at_M_Chinnaswamy_Stadium', 'data': {'Batting': {'Matches': 109, 'Innings': 106, 'Runs': 3618, 'Balls': 2514, 'Outs': 92, '4s': 329, '6s': 154, '50s': 27, '100s': 4, 'SR': 143.91, 'Avg': 39.32}}}], 'head_2_head_stats': [{'opponent': 'Jasprit Bumrah', 'opp_role': 'bowler', 'stats': {'Title': 'Virat Kohli V/S Jasprit Bumrah', 'Stats': {'Innings': '17', 'Runs': '150', 'Balls': '101', 'Outs': '5', 'Dots

In [ ]:
example = {'Title': 'Virat Kohli V/S Jasprit Bumrah', 'Stats': {'Innings': '17', 'Runs': '150', 'Balls': '101', 'Outs': '12', 'Dots': '37', '4s': '15', '6s': '6', 'SR': '148.5', 'Avg': '30.0'}}

result = compute_faceoff_score(example)
print(result)


{'Title': 'Virat Kohli V/S Jasprit Bumrah', 'Stats': {'Innings': '17', 'Runs': '150', 'Balls': '101', 'Outs': '12', 'Dots': '37', '4s': '15', '6s': '6', 'SR': '148.5', 'Avg': '30.0'}, 'advantage_score': -0.01}


In [ ]:
 # to do -> captain, vice-captain, batting position or bowling time, minInn cond for recent_stats, opp, ground
# complete name 
# use a separate tool just for role


In [10]:
stats = get_player_pace_spin_stats("rohit sharma")
print(stats["pace"])  # Aggregated pace‐bowling stats
print(stats["spin"])  # Aggregated spin‐bowling stats


{'Runs': 7751, 'Balls': 5404, 'Outs': 247, '4s': 777, '6s': 367, '50s': 13, '100s': 0, 'SR': 143.43, 'Avg': 31.38}
{'Runs': 3826, 'Balls': 3169, 'Outs': 115, '4s': 269, '6s': 156, '50s': 0, '100s': 0, 'SR': 120.73, 'Avg': 33.27}


In [4]:
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

model = ChatOpenAI(model="gpt-4o")

# Create specialized agents

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

research_agent = create_react_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)
config = {"configurable": {"thread_id": "123"}}
# Compile and run
app = workflow.compile(checkpointer=memory)

In [5]:
while True:
    inp = input("Enter: ")
    if inp == "exit":
        break
    result = app.invoke({
        "messages": [
            {
                "role": "user",
                "content": inp
            }
        ]
    }, config = config)
    for a in result["messages"]:
        a.pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================
Name: supervisor

Hello! How can I assist you today?
================================ Human Message =================================

hi
================================== Ai Message ==================================
Name: supervisor

Hello! How can I assist you today?
================================ Human Message =================================

who are you
================================== Ai Message ==================================
Name: supervisor

I am an AI assistant here to help you with research questions, math problems, and other information-related queries. How can I assist you today?
================================ Human Message =================================

hi
================================== Ai Message ==================================
Name: supervisor

Hello! How can I assist you today?
=====

In [13]:
def check_weather(location: str) -> str:
        '''Return the weather forecast for the specified location.'''
        return f"It's always sunny in {location}"

graph = create_react_agent(
        model = LLM,
        tools=[check_weather],
        prompt="You are a helpful assistant",   
    )
inputs = {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
for chunk in graph.stream(inputs, stream_mode="updates", subgraphs=True):
    print(chunk)

((), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ciedFNDNlsu19rwhuZQIdJ7x', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'check_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 58, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b3f1157249', 'id': 'chatcmpl-BgNh5LcwxJ2ut0nzp4r0IrPHMBA2r', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--54b41921-64cb-4648-b80a-3cdfa6886471-0', tool_calls=[{'name': 'check_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_ciedFNDNlsu19rwhuZQIdJ7x', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 15

In [17]:
        
        
stream_modes_type = """
        - `"values"`: Emit all values in the state after each step, including interrupts.
            When used with functional API, values are emitted once at the end of the workflow.
        - `"updates"`: Emit only the node or task names and updates returned by the nodes or tasks after each step.
            If multiple updates are made in the same step (e.g. multiple nodes are run) then those updates are emitted separately.
        - `"custom"`: Emit custom data from inside nodes or tasks using `StreamWriter`.
        - `"messages"`: Emit LLM messages token-by-token together with metadata for any LLM invocations inside nodes or tasks.
            Will be emitted as 2-tuples `(LLM token, metadata)`.
        - `"debug"`: Emit debug events with as much information as possible for each step.
        """